In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import warnings
from collections import Counter
import re
import requests
import zipfile
from io import BytesIO
import shutil
import json

warnings.filterwarnings('ignore')

# Create directories if they don't exist
os.makedirs('data/raw/liar_dataset', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('results/plots', exist_ok=True)
os.makedirs('results/reports', exist_ok=True)

print("="*60)
print("🎭 BIAS ANALYSIS & ADVANCED VISUALIZATIONS")
print("Member: ITBIN-2211-0184")
print("Time: 2:00 PM - 4:00 PM")
print("="*60)

# Download and extract the dataset if missing
def download_dataset():
    dataset_url = "https://raw.githubusercontent.com/msantinelli/fake-news-detection/main/data/raw/liar_dataset.zip"
    print(f"⚠️ Dataset not found. Downloading from {dataset_url}...")
    
    try:
        response = requests.get(dataset_url)
        response.raise_for_status()  # Check for HTTP errors
        
        with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
            # Extract directly to data/raw directory
            zip_ref.extractall('data/raw')
            print("✅ Dataset downloaded and extracted successfully")
            
            # Move files to liar_dataset directory
            for file in ['train.tsv', 'test.tsv', 'valid.tsv']:
                src = f'data/raw/{file}'
                dst = f'data/raw/liar_dataset/{file}'
                if os.path.exists(src):
                    shutil.move(src, dst)
            
            print("📁 Files moved to data/raw/liar_dataset/")
            
        return True
    except Exception as e:
        print(f"❌ Error downloading dataset: {e}")
        return False

# Load the dataset from raw files
def load_raw_dataset():
    # Define column names for LIAR dataset
    columns = [
        'id', 'label', 'statement', 'subject', 'speaker', 'speaker_job', 
        'state_info', 'party_affiliation', 
        'barely_true_counts', 'false_counts', 'half_true_counts', 
        'mostly_true_counts', 'pants_on_fire_counts'
    ]
    
    # Check if files exist, download if missing
    required_files = [
        'data/raw/liar_dataset/train.tsv',
        'data/raw/liar_dataset/test.tsv',
        'data/raw/liar_dataset/valid.tsv'
    ]
    
    if not all(os.path.exists(f) for f in required_files):
        if not download_dataset():
            raise FileNotFoundError("Could not load dataset files")
    
    # Load train, test and validation datasets
    try:
        train_df = pd.read_csv('data/raw/liar_dataset/train.tsv', sep='\t', header=None, names=columns)
        test_df = pd.read_csv('data/raw/liar_dataset/test.tsv', sep='\t', header=None, names=columns)
        valid_df = pd.read_csv('data/raw/liar_dataset/valid.tsv', sep='\t', header=None, names=columns)
        
        # Add split information
        train_df['split'] = 'train'
        test_df['split'] = 'test'
        valid_df['split'] = 'valid'
        
        # Combine datasets
        df = pd.concat([train_df, test_df, valid_df], ignore_index=True)
        print(f"✅ Loaded raw dataset: {df.shape}")
        
        # Minimal preprocessing
        df['text_length'] = df['statement'].apply(len)
        df['party_affiliation'] = df['party_affiliation'].fillna('unknown').str.lower().str.strip()
        df['subject'] = df['subject'].fillna('unknown').str.lower().str.strip()
        df['speaker_job'] = df['speaker_job'].fillna('unknown')
        
        # Create simplified labels
        label_mapping = {
            'true': 'true',
            'mostly-true': 'mostly-true',
            'half-true': 'half-true',
            'barely-true': 'barely-true',
            'false': 'false',
            'pants-fire': 'pants-fire'
        }
        df['label'] = df['label'].map(label_mapping).fillna('unknown')
        
        # Save processed data for future use
        df.to_csv('data/processed/liar_dataset_processed.csv', index=False)
        print("💾 Saved processed data to 'data/processed/liar_dataset_processed.csv'")
        
        return df
    except Exception as e:
        print(f"❌ Error loading raw data: {e}")
        raise

# Try to load processed data
try:
    df = pd.read_csv('data/processed/liar_dataset_processed.csv')
    print(f"✅ Loaded processed dataset: {df.shape}")
except FileNotFoundError:
    df = load_raw_dataset()

def comprehensive_political_bias_analysis(df):
    """Perform comprehensive political bias analysis"""
    print("\n" + "="*50)
    print("🏛️ COMPREHENSIVE POLITICAL BIAS ANALYSIS")
    print("="*50)
    
    bias_analysis = {}
    
    # 1. Truth rate by party affiliation
    print("1️⃣ Truth Rate Analysis by Party Affiliation")
    print("-" * 45)
    
    # Define truth categories
    high_truth = ['true', 'mostly-true']
    low_truth = ['false', 'pants-fire']
    
    # Calculate truth rates by party
    party_stats = []
    top_parties = df['party_affiliation'].value_counts().head(10).index
    
    for party in top_parties:
        party_data = df[df['party_affiliation'] == party]
        total_statements = len(party_data)
        
        high_truth_count = party_data['label'].isin(high_truth).sum()
        low_truth_count = party_data['label'].isin(low_truth).sum()
        
        high_truth_rate = (high_truth_count / total_statements) * 100
        low_truth_rate = (low_truth_count / total_statements) * 100
        
        party_stats.append({
            'party': party,
            'total_statements': total_statements,
            'high_truth_rate': high_truth_rate,
            'low_truth_rate': low_truth_rate,
            'high_truth_count': high_truth_count,
            'low_truth_count': low_truth_count
        })
        
        print(f"{party:20} | {total_statements:4d} statements | Truth: {high_truth_rate:5.1f}% | False: {low_truth_rate:5.1f}%")
    
    party_df = pd.DataFrame(party_stats)
    bias_analysis['party_truth_rates'] = party_df
    
    # 2. Subject matter bias analysis
    print(f"\n2️⃣ Subject Matter Bias Analysis")
    print("-" * 32)
    
    subject_stats = []
    top_subjects = df['subject'].value_counts().head(10).index
    
    for subject in top_subjects:
        subject_data = df[df['subject'] == subject]
        total_statements = len(subject_data)
        
        high_truth_count = subject_data['label'].isin(high_truth).sum()
        high_truth_rate = (high_truth_count / total_statements) * 100
        
        subject_stats.append({
            'subject': subject,
            'total_statements': total_statements,
            'high_truth_rate': high_truth_rate,
        })
        
        print(f"{subject:25} | {total_statements:4d} statements | Truth: {high_truth_rate:5.1f}%")
    
    subject_df = pd.DataFrame(subject_stats)
    bias_analysis['subject_truth_rates'] = subject_df
    
    # 3. Speaker role bias analysis
    print(f"\n3️⃣ Speaker Role Bias Analysis")
    print("-" * 28)
    
    # Clean and analyze speaker jobs
    df['speaker_job_clean'] = df['speaker_job'].fillna('Unknown').str.title()
    
    role_stats = []
    top_roles = df['speaker_job_clean'].value_counts().head(10).index
    
    for role in top_roles:
        role_data = df[df['speaker_job_clean'] == role]
        total_statements = len(role_data)
        
        if total_statements >= 10:  # Only analyze roles with sufficient data
            high_truth_count = role_data['label'].isin(high_truth).sum()
            high_truth_rate = (high_truth_count / total_statements) * 100
            
            role_stats.append({
                'role': role,
                'total_statements': total_statements,
                'high_truth_rate': high_truth_rate,
            })
            
            print(f"{role:25} | {total_statements:4d} statements | Truth: {high_truth_rate:5.1f}%")
    
    role_df = pd.DataFrame(role_stats)
    bias_analysis['role_truth_rates'] = role_df
    
    # 4. Temporal bias analysis (by data split as proxy)
    print(f"\n4️⃣ Data Split Analysis")
    print("-" * 22)
    
    split_stats = []
    for split in df['split'].unique():
        split_data = df[df['split'] == split]
        total_statements = len(split_data)
        
        high_truth_count = split_data['label'].isin(high_truth).sum()
        high_truth_rate = (high_truth_count / total_statements) * 100
        
        split_stats.append({
            'split': split,
            'total_statements': total_statements,
            'high_truth_rate': high_truth_rate,
        })
        
        print(f"{split:10} | {total_statements:4d} statements | Truth: {high_truth_rate:5.1f}%")
    
    bias_analysis['split_analysis'] = pd.DataFrame(split_stats)
    
    return bias_analysis

def create_political_bias_visualizations(df, bias_analysis):
    """Create comprehensive political bias visualizations"""
    print("\n📊 Creating political bias visualizations...")
    
    # Create large figure with multiple subplots
    fig, axes = plt.subplots(3, 3, figsize=(22, 18))
    fig.suptitle('Comprehensive Political Bias Analysis', fontsize=18, fontweight='bold')
    
    # 1. Party truth rates (horizontal bar chart)
    party_df = bias_analysis['party_truth_rates'].sort_values('high_truth_rate', ascending=True)
    colors = plt.cm.RdYlBu_r(np.linspace(0.2, 0.8, len(party_df)))
    
    bars = axes[0,0].barh(range(len(party_df)), party_df['high_truth_rate'], color=colors)
    axes[0,0].set_yticks(range(len(party_df)))
    axes[0,0].set_yticklabels(party_df['party'], fontsize=10)
    axes[0,0].set_title('Truth Rate by Party Affiliation', fontweight='bold')
    axes[0,0].set_xlabel('High Truth Rate (%)')
    
    # Add value labels
    for i, (bar, rate) in enumerate(zip(bars, party_df['high_truth_rate'])):
        axes[0,0].text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
                      f'{rate:.1f}%', va='center', fontsize=9)
    
    # 2. Subject matter truth rates
    subject_df = bias_analysis['subject_truth_rates'].sort_values('high_truth_rate', ascending=True)
    colors = plt.cm.Greens(np.linspace(0.3, 0.9, len(subject_df)))
    
    bars = axes[0,1].barh(range(len(subject_df)), subject_df['high_truth_rate'], color=colors)
    axes[0,1].set_yticks(range(len(subject_df)))
    axes[0,1].set_yticklabels([s[:20] + '...' if len(s) > 20 else s for s in subject_df['subject']], fontsize=9)
    axes[0,1].set_title('Truth Rate by Subject Matter', fontweight='bold')
    axes[0,1].set_xlabel('High Truth Rate (%)')
    
    # Add value labels
    for i, (bar, rate) in enumerate(zip(bars, subject_df['high_truth_rate'])):
        axes[0,1].text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
                      f'{rate:.1f}%', va='center', fontsize=9)
    
    # 3. Speaker role analysis
    if len(bias_analysis['role_truth_rates']) > 0:
        role_df = bias_analysis['role_truth_rates'].sort_values('high_truth_rate', ascending=True)
        colors = plt.cm.Oranges(np.linspace(0.3, 0.9, len(role_df)))
        
        bars = axes[0,2].barh(range(len(role_df)), role_df['high_truth_rate'], color=colors)
        axes[0,2].set_yticks(range(len(role_df)))
        axes[0,2].set_yticklabels([r[:15] + '...' if len(r) > 15 else r for r in role_df['role']], fontsize=9)
        axes[0,2].set_title('Truth Rate by Speaker Role', fontweight='bold')
        axes[0,2].set_xlabel('High Truth Rate (%)')
    else:
        axes[0,2].axis('off')
    
    # 4. Party vs Subject heatmap (truth rates)
    party_subject_pivot = df.groupby(['party_affiliation', 'subject']).apply(
        lambda x: (x['label'].isin(['true', 'mostly-true']).sum() / len(x)) * 100
    ).reset_index(name='truth_rate')
    
    # Filter for top parties and subjects
    top_parties = df['party_affiliation'].value_counts().head(6).index
    top_subjects = df['subject'].value_counts().head(6).index
    
    pivot_filtered = party_subject_pivot[
        (party_subject_pivot['party_affiliation'].isin(top_parties)) &
        (party_subject_pivot['subject'].isin(top_subjects))
    ]
    
    if not pivot_filtered.empty:
        pivot_table = pivot_filtered.pivot(index='party_affiliation', columns='subject', values='truth_rate')
        sns.heatmap(pivot_table, annot=True, cmap='RdYlGn', center=50, ax=axes[1,0], fmt='.1f')
        axes[1,0].set_title('Truth Rate Heatmap: Party vs Subject', fontweight='bold')
        axes[1,0].set_ylabel('Party Affiliation')
        axes[1,0].set_xlabel('Subject')
    else:
        axes[1,0].text(0.5, 0.5, 'Insufficient data for heatmap', 
                       ha='center', va='center', fontsize=12)
        axes[1,0].set_title('Truth Rate Heatmap: Party vs Subject', fontweight='bold')
    
    # 5. Statement distribution by party
    party_counts = df['party_affiliation'].value_counts().head(8)
    axes[1,2].pie(party_counts.values, labels=[p[:10] + '...' if len(p) > 10 else p for p in party_counts.index],
                  autopct='%1.1f%%', startangle=90)
    axes[1,2].set_title('Statement Distribution by Party', fontweight='bold')
    
    # 6. Truth rate distribution across all parties
    all_party_truth_rates = []
    for party in df['party_affiliation'].unique():
        party_data = df[df['party_affiliation'] == party]
        if len(party_data) >= 5:  # Minimum statements for reliable rate
            truth_rate = (party_data['label'].isin(['true', 'mostly-true']).sum() / len(party_data)) * 100
            all_party_truth_rates.append(truth_rate)
    
    if all_party_truth_rates:
        axes[2,0].hist(all_party_truth_rates, bins=15, alpha=0.7, color='skyblue', edgecolor='black')
        axes[2,0].set_title('Distribution of Truth Rates Across Parties', fontweight='bold')
        axes[2,0].set_xlabel('Truth Rate (%)')
        axes[2,0].set_ylabel('Number of Parties')
        axes[2,0].axvline(np.mean(all_party_truth_rates), color='red', linestyle='--', label=f'Mean: {np.mean(all_party_truth_rates):.1f}%')
        axes[2,0].legend()
    else:
        axes[2,0].text(0.5, 0.5, 'Insufficient party data', 
                       ha='center', va='center', fontsize=12)
        axes[2,0].set_title('Distribution of Truth Rates Across Parties', fontweight='bold')
    
    # 7. Subject matter popularity vs truth rate
    if 'subject_truth_rates' in bias_analysis:
        subject_df = bias_analysis['subject_truth_rates']
        axes[2,1].scatter(subject_df['total_statements'], subject_df['high_truth_rate'],
                         s=60, alpha=0.7, c='green')
        
        # Add subject labels for interesting points
        for _, row in subject_df.iterrows():
            if row['total_statements'] > 100 or row['high_truth_rate'] > 70 or row['high_truth_rate'] < 30:
                axes[2,1].annotate(row['subject'][:15], xy=(row['total_statements'], row['high_truth_rate']), xytext=(5, 5), textcoords='offset points', fontsize=8)
        
        axes[2,1].set_xlabel('Number of Statements')
        axes[2,1].set_ylabel('Truth Rate (%)')
        axes[2,1].set_title('Subject Popularity vs Truth Rate', fontweight='bold')
    else:
        axes[2,1].text(0.5, 0.5, 'No subject data', 
                       ha='center', va='center', fontsize=12)
        axes[2,1].set_title('Subject Popularity vs Truth Rate', fontweight='bold')
    
    # 8. Bias summary metrics
    # Calculate bias metrics
    party_df = bias_analysis['party_truth_rates']
    truth_rate_std = party_df['high_truth_rate'].std()
    truth_rate_range = party_df['high_truth_rate'].max() - party_df['high_truth_rate'].min()
    
    metrics_text = f"""BIAS METRICS SUMMARY

Truth Rate Standard Deviation: {truth_rate_std:.1f}%
Truth Rate Range: {truth_rate_range:.1f}%

Most Truthful Party:
{party_df.loc[party_df['high_truth_rate'].idxmax(), 'party'][:20]}
({party_df['high_truth_rate'].max():.1f}%)

Least Truthful Party:
{party_df.loc[party_df['high_truth_rate'].idxmin(), 'party'][:20]}
({party_df['high_truth_rate'].min():.1f}%)

Total Parties Analyzed: {len(party_df)}
Total Statements: {party_df['total_statements'].sum():,}
"""
    
    axes[2,2].text(0.1, 0.9, metrics_text, transform=axes[2,2].transAxes, fontsize=11,
                   verticalalignment='top', fontfamily='monospace',
                   bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
    axes[2,2].set_xlim(0, 1)
    axes[2,2].set_ylim(0, 1)
    axes[2,2].axis('off')
    axes[2,2].set_title('Bias Analysis Summary', fontweight='bold')
    
    # Hide unused subplot
    axes[1,1].axis('off')
    
    plt.tight_layout()
    plt.savefig('results/plots/day2_political_bias_analysis.png', dpi=300, bbox_inches='tight')
    print("✅ Political bias analysis saved to 'results/plots/day2_political_bias_analysis.png'")
    plt.show()

# Perform comprehensive political bias analysis
bias_results = comprehensive_political_bias_analysis(df)

# Create political bias visualizations
create_political_bias_visualizations(df, bias_results)

def create_advanced_text_visualizations(df):
    """Create advanced text analysis and word cloud visualizations"""
    print("\n" + "="*50)
    print("📝 ADVANCED TEXT VISUALIZATIONS")
    print("="*50)
    
    # Prepare text data
    df['statement_clean'] = df['statement'].fillna('').astype(str)
    
    # Create figure for text visualizations
    fig, axes = plt.subplots(2, 3, figsize=(20, 14))
    fig.suptitle('Advanced Text Analysis Visualizations', fontsize=16, fontweight='bold')
    
    # 1. Word clouds by truth label
    print("🌤️ Creating word clouds by truth label...")
    
    # True statements word cloud
    true_text = ' '.join(df[df['label'] == 'true']['statement_clean'].values)
    if len(true_text.strip()) > 0:
        wordcloud_true = WordCloud(width=400, height=300, background_color='white',
                                  colormap='Greens', max_words=50).generate(true_text)
        axes[0,0].imshow(wordcloud_true, interpolation='bilinear')
        axes[0,0].set_title('Word Cloud: TRUE Statements', fontweight='bold')
        axes[0,0].axis('off')
    else:
        axes[0,0].text(0.5, 0.5, 'No true statements', 
                       ha='center', va='center', fontsize=12)
        axes[0,0].set_title('Word Cloud: TRUE Statements', fontweight='bold')
    
    # False statements word cloud
    false_text = ' '.join(df[df['label'] == 'false']['statement_clean'].values)
    if len(false_text.strip()) > 0:
        wordcloud_false = WordCloud(width=400, height=300, background_color='white',
                                   colormap='Reds', max_words=50).generate(false_text)
        axes[0,1].imshow(wordcloud_false, interpolation='bilinear')
        axes[0,1].set_title('Word Cloud: FALSE Statements', fontweight='bold')
        axes[0,1].axis('off')
    else:
        axes[0,1].text(0.5, 0.5, 'No false statements', 
                       ha='center', va='center', fontsize=12)
        axes[0,1].set_title('Word Cloud: FALSE Statements', fontweight='bold')
    
    # Pants-fire statements word cloud
    pants_fire_text = ' '.join(df[df['label'] == 'pants-fire']['statement_clean'].values)
    if len(pants_fire_text.strip()) > 0:
        wordcloud_pants = WordCloud(width=400, height=300, background_color='white',
                                   colormap='OrRd', max_words=50).generate(pants_fire_text)
        axes[0,2].imshow(wordcloud_pants, interpolation='bilinear')
        axes[0,2].set_title('Word Cloud: PANTS-FIRE Statements', fontweight='bold')
        axes[0,2].axis('off')
    else:
        axes[0,2].text(0.5, 0.5, 'No pants-fire statements', 
                       ha='center', va='center', fontsize=12)
        axes[0,2].set_title('Word Cloud: PANTS-FIRE Statements', fontweight='bold')
    
    # 2. Most common words analysis
    print("📊 Analyzing most common words...")
    
    # Extract common words for each category
    def extract_common_words(text_series, top_n=15):
        all_text = ' '.join(text_series.fillna('').astype(str))
        # Simple word extraction
        words = re.findall(r'\b[a-zA-Z]{3,}\b', all_text.lower())
        # Remove common stop words
        stop_words = {'the', 'and', 'that', 'have', 'for', 'not', 'with', 'you', 'this', 
                     'but', 'his', 'from', 'they', 'she', 'her', 'been', 'than', 'its', 
                     'are', 'was', 'will', 'would', 'could', 'should', 'said', 'says'}
        words = [w for w in words if w not in stop_words and len(w) > 3]
        return Counter(words).most_common(top_n)
    
    # True statements common words
    true_words = extract_common_words(df[df['label'] == 'true']['statement_clean'])
    if true_words:
        words, counts = zip(*true_words[:10])
        axes[1,0].barh(range(len(words)), counts, color='green', alpha=0.7)
        axes[1,0].set_yticks(range(len(words)))
        axes[1,0].set_yticklabels(words)
        axes[1,0].set_title('Most Common Words: TRUE Statements', fontweight='bold')
        axes[1,0].set_xlabel('Frequency')
    else:
        axes[1,0].text(0.5, 0.5, 'No true statements', 
                       ha='center', va='center', fontsize=12)
        axes[1,0].set_title('Most Common Words: TRUE Statements', fontweight='bold')
    
    # False statements common words
    false_words = extract_common_words(df[df['label'] == 'false']['statement_clean'])
    if false_words:
        words, counts = zip(*false_words[:10])
        axes[1,1].barh(range(len(words)), counts, color='red', alpha=0.7)
        axes[1,1].set_yticks(range(len(words)))
        axes[1,1].set_yticklabels(words)
        axes[1,1].set_title('Most Common Words: FALSE Statements', fontweight='bold')
        axes[1,1].set_xlabel('Frequency')
    else:
        axes[1,1].text(0.5, 0.5, 'No false statements', 
                       ha='center', va='center', fontsize=12)
        axes[1,1].set_title('Most Common Words: FALSE Statements', fontweight='bold')
    
    # 3. Text length patterns
    print("📏 Analyzing text length patterns...")
    
    # Create text length distribution by label
    labels_for_length = ['true', 'false', 'mostly-true', 'barely-true']
    colors = ['green', 'red', 'blue', 'orange']
    
    has_data = False
    for label, color in zip(labels_for_length, colors):
        if label in df['label'].values:
            lengths = df[df['label'] == label]['text_length']
            axes[1,2].hist(lengths, bins=20, alpha=0.6, label=label, color=color)
            has_data = True
    
    if has_data:
        axes[1,2].set_xlabel('Text Length (characters)')
        axes[1,2].set_ylabel('Frequency')
        axes[1,2].set_title('Text Length Distribution by Label', fontweight='bold')
        axes[1,2].legend()
        axes[1,2].set_xlim(0, 500)  # Focus on reasonable range
    else:
        axes[1,2].text(0.5, 0.5, 'No text length data', 
                       ha='center', va='center', fontsize=12)
        axes[1,2].set_title('Text Length Distribution by Label', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('results/plots/day2_advanced_text_visualizations.png', dpi=300, bbox_inches='tight')
    print("✅ Advanced text visualizations saved to 'results/plots/day2_advanced_text_visualizations.png'")
    plt.show()

def create_interactive_dashboard_data(df, bias_results):
    """Prepare data for interactive dashboard creation"""
    print("\n🎛️ Preparing interactive dashboard data...")
    
    # Create summary statistics for dashboard
    dashboard_data = {
        'overview': {
            'total_statements': len(df),
            'unique_speakers': df['speaker'].nunique(),
            'unique_subjects': df['subject'].nunique(),
            'unique_parties': df['party_affiliation'].nunique(),
            'truth_rate_overall': (df['label'].isin(['true', 'mostly-true']).sum() / len(df)) * 100
        },
        'party_analysis': bias_results['party_truth_rates'].to_dict('records'),
        'subject_analysis': bias_results['subject_truth_rates'].to_dict('records'),
        'label_distribution': df['label'].value_counts().to_dict(),
        'text_stats': {
            'avg_length': df['text_length'].mean(),
            'length_by_label': df.groupby('label')['text_length'].mean().to_dict()
        }
    }
    
    # Save dashboard data
    with open('results/reports/dashboard_data.json', 'w') as f:
        json.dump(dashboard_data, f, indent=2, default=str)
    
    print("✅ Dashboard data saved to 'results/reports/dashboard_data.json'")
    
    return dashboard_data

# Create advanced text visualizations
create_advanced_text_visualizations(df)

# Prepare interactive dashboard data
dashboard_data = create_interactive_dashboard_data(df, bias_results)

def generate_final_day2_summary(df, bias_results, dashboard_data):
    """Generate final comprehensive Day 2 summary"""
    print("\n" + "="*60)
    print("📋 FINAL DAY 2 COMPREHENSIVE SUMMARY")
    print("="*60)
    
    print("🎯 ACHIEVEMENTS TODAY:")
    print("   ✅ Advanced linguistic feature extraction")
    print("   ✅ Comprehensive statistical significance testing")
    print("   ✅ Detailed political bias analysis")
    print("   ✅ Advanced text visualizations and word clouds")
    print("   ✅ Interactive dashboard data preparation")
    
    print(f"\n📊 KEY INSIGHTS DISCOVERED:")
    
    # Political bias insights
    party_df = bias_results['party_truth_rates']
    if not party_df.empty:
        max_truth_party = party_df.loc[party_df['high_truth_rate'].idxmax()]
        min_truth_party = party_df.loc[party_df['high_truth_rate'].idxmin()]
        
        print(f"   🏛️ Political Bias:")
        print(f"      • Highest truth rate: {max_truth_party['party']} ({max_truth_party['high_truth_rate']:.1f}%)")
        print(f"      • Lowest truth rate: {min_truth_party['party']} ({min_truth_party['high_truth_rate']:.1f}%)")
        print(f"      • Truth rate spread: {party_df['high_truth_rate'].max() - party_df['high_truth_rate'].min():.1f} percentage points")
    else:
        print("   🏛️ Political Bias: No party data available")
    
    # Subject matter insights
    subject_df = bias_results['subject_truth_rates']
    if not subject_df.empty:
        max_truth_subject = subject_df.loc[subject_df['high_truth_rate'].idxmax()]
        min_truth_subject = subject_df.loc[subject_df['high_truth_rate'].idxmin()]
        
        print(f"   📚 Subject Matter Bias:")
        print(f"      • Most truthful subject: {max_truth_subject['subject'][:30]}... ({max_truth_subject['high_truth_rate']:.1f}%)")
        print(f"      • Least truthful subject: {min_truth_subject['subject'][:30]}... ({min_truth_subject['high_truth_rate']:.1f}%)")
    else:
        print("   📚 Subject Matter Bias: No subject data available")
    
    # Text characteristics
    print(f"   📝 Text Characteristics:")
    print(f"      • Average statement length: {df['text_length'].mean():.0f} characters")
    
    print(f"\n💾 FILES GENERATED TODAY:")
    files_generated = [
        "results/plots/day2_political_bias_analysis.png",
        "results/plots/day2_advanced_text_visualizations.png",
        "results/reports/dashboard_data.json",
        "data/processed/liar_dataset_processed.csv"
    ]
    
    for file in files_generated:
        print(f"   📄 {file}")
    
    print(f"\n🔄 PREPARATION FOR DAY 3:")
    print("   📋 Data profiling reports ready")
    print("   📊 Advanced visualizations completed")
    print("   🎛️ Dashboard data prepared")
    print("   📈 Statistical analysis comprehensive")
    print("   🏛️ Bias analysis thorough")
    
    print(f"\n⏰ DAY 2 TIMELINE COMPLETED:")
    print("   ✅ 9:00-11:00 AM: Advanced EDA & Linguistic Analysis")
    print("   ✅ 11:00-1:00 PM: Statistical Testing & Correlation Analysis")
    print("   ✅ 2:00-4:00 PM: Political Bias & Advanced Visualizations")
    
    return {
        'completion_status': 'Day 2 Complete',
        'insights_generated': 4,
        'files_generated': len(files_generated),
        'next_phase': 'Day 3: Final Documentation & Reporting'
    }


⚠️ Processed data not found. Loading raw data...
❌ Error loading raw data: [Errno 2] No such file or directory: 'data/raw/liar_dataset/train.tsv'
Please download the LIAR dataset and place it in 'data/raw/liar_dataset/'
Dataset available at: https://www.cs.ucsb.edu/~william/data/liar_dataset.zip


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/liar_dataset/train.tsv'